KHAI BÁO THƯ VIỆN



In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np 
import pandas as pd 
import io 
import requests
import datetime as dt


Các Hàm chuyển đổi đơn vị

In [2]:

#Công thức chuyển từ PPM thành mg/m^3
def ConvertAirFrom_PPM_To_mgDivm3(x):
    return x*(0.0409)*44
#Công thức chuyển từ mg/m^3 thành micro mol/m^3
def ConvertAirFrom_mgDivm3_To_UmolDivm3(x):
    return x*(1000/44)
#Công thức chuyển từ micromol/m^3 thành mg/m^3
def ConvertAirFrom_UmolDivm3_To_mgDivm3(x):
    return x/(1000/44)
#Công thức chuyển từ mg/m^3 thành micro mol/m^3
def ConvertAirFrom_mgDivm3_To_UmolDivmol(x):
    return x/((0.0409)*44)


HÀM TÍNH GIÁ TRỊ CALLABLE DX

In [3]:
def dx(cap_CO2_Air,cap_CO2_Top,MC_BlowAir,MC_ExtAir,MC_PadAir,MC_AirCan,MC_AirTop,MC_AirOut,MC_TopOut):
    global CO2_Air
    MC_BlowAir=def_MC_BlowAir(Eta_HeatCO2,U_Blow,P_Blow,A_Flr)

    MC_ExtAir=def_MC_ExtAir(U_ExtCO2,Phi_ExtCO2,A_Flr)

    MC_PadAir=def_MC_PadAir(U_Pad,Phi_Pad,A_Flr,CO2_Out,CO2_Air)


    f_ThScr=def_f_ThScr(U_ThScr,K_ThScr,Delta_T_AirTop,g,RO_Mean,Delta_RO)
    MC_AirTop=def_MC_AirTop(f_ThScr,CO2_Air,CO2_Top)


    ff_VentSide=def_f_VentRoofSide(C_d,A_Flr,U_Roof,0,U_Side,A_Side,g,h_SideRoof,Delta_T_AirOut,T_mean,C_w,v_Wind) # A_Roof=0
    Eta_InsScr=def_Eta_InsScr(Zeta_InsScr)
    f_leakage=def_f_leakage(v_Wind,C_leakage)
    f_VentRoofSide=def_f_VentRoofSide(C_d,A_Flr,U_Roof,A_Roof,U_Side,A_Side,g,h_SideRoof,Delta_T_AirOut,T_mean,C_w,v_Wind)
    f_VentForced=def_f_VentForced(Eta_InsScr,U_VentForced,Phi_VentForced,A_Flr)
    f_VentSide=def_f_VentSide(Eta_InsScr,ff_VentSide,f_leakage,U_ThScr,f_VentRoofSide,Eta_Side,Eta_SideThr)
    MC_AirOut=def_MC_AirOut(f_VentSide,f_VentForced,CO2_Air,CO2_Out)
    
    ff_VentRoof=def_ff_VentRoof(C_d,U_Roof,A_Roof,A_Flr,g,h_Roof,Delta_T_AirOut,T_mean,C_w,v_Wind)
    f_VentRoof=def_f_VentRoof(Eta_InsScr,ff_VentRoof,f_leakage,U_ThScr,f_VentRoofSide,Eta_Roof,Eta_RoofThr) 

    MC_TopOut=def_MC_TopOut(f_VentRoof,CO2_Top,CO2_Out)

    h_C_Buf=def_h_C_Buf(C_Buf,C_Max_Buf)

    #Phần tốc độ phản ứng
    #Đổi đơn vị của CO2
    CO2_Air=ConvertAirFrom_mgDivm3_To_UmolDivmol(float(CO2_Air))

    Gamma=def_Gamma(c_Gamma,T_Can)
    CO2_Stom=def_CO2_Stom(Eta_CO2_Air_Stom,CO2_Air)
    J_Max_Can=def_J_Max_Can(LAI,J_Max_Leaf)
    J_POT=def_J_POT(J_Max_Can,E_j,T_Can,T_25,R,S,H)
    J=def_J(J_POT,anpha,PAR_Can,Theta)
    P=def_P(J,CO2_Stom,Gamma)
    R_HOHAP=0.01*P
    MC_AirCan=def_MC_AirCan(M_CH2O,h_C_Buf,P,R_HOHAP)

    #đổi lại đơn vị cho CO2_Air
    CO2_Air=ConvertAirFrom_PPM_To_mgDivm3(float(CO2_Air))

    dx1=(MC_BlowAir+MC_ExtAir+MC_PadAir-MC_AirCan-MC_AirTop-MC_AirOut)/cap_CO2_Air
    dx2=(MC_AirTop-MC_TopOut)/cap_CO2_Top
    return [dx1,dx2]

CÁC PHƯƠNG TRÌNH TÍNH TỪNG BIỂU THỨC

DÙNG BIẾN TOÀN CỤC ĐỂ LƯU GIÁ TRỊ CÁC GIÁ TRỊ TRẢ VỀ Ở MỖI BIỂU THỨC ĐỂ ÁP DỤNG CHO CÔNG THỨC TIẾP THEO

In [4]:
#Phương trình 3
def def_MC_BlowAir(Eta_HeatCO2,U_Blow,P_Blow,A_Flr):
    return Eta_HeatCO2*U_Blow*P_Blow/A_Flr

#Phương trình 4
def def_MC_ExtAir(U_ExtCO2,Phi_ExtCO2,A_Flr):
    return U_ExtCO2*Phi_ExtCO2/A_Flr

#Phương trình 5
def def_MC_PadAir(U_Pad,Phi_Pad,A_Flr,CO2_Out,CO2_Air):
    return U_Pad*Phi_Pad*(CO2_Out-CO2_Air)/A_Flr

#Phương trình 7
def def_f_ThScr(U_ThScr,K_ThScr,Delta_T_AirTop,g,RO_Mean,Delta_RO):
    return U_ThScr*K_ThScr*((Delta_T_AirTop)**(2/3))+(1-U_ThScr)*(((g*(1-U_ThScr)/(2*RO_Mean))*Delta_RO)**(1/2))

#Phương trình 6
def def_MC_AirTop(f_ThScr,CO2_Air,CO2_Top):
    return f_ThScr*(CO2_Air-CO2_Top)

#Phương trình 10
def def_f_VentRoofSide(C_d,A_Flr,U_Roof,A_Roof,U_Side,A_Side,g,h_SideRoof,Delta_T_AirOut,T_mean,C_w,v_Wind):
    component1=C_d/A_Flr
    component2=((U_Roof**2)*(U_Side**2)*(A_Roof**2)*(A_Side**2))/((U_Roof**2)*(A_Roof**2)+(U_Side**2)*(A_Side**2))
    component3=(2*g*h_SideRoof*Delta_T_AirOut)/T_mean
    component4=(((U_Roof*A_Roof+U_Side*A_Side)/2)**2)*C_w*(v_Wind**2)
    return component1*((component2*component3+component4)**(1/2))

#Phương trình 11
def def_Eta_InsScr(Zeta_InsScr):
    return Zeta_InsScr*(2-Zeta_InsScr)

#Phương trình 12
def def_f_leakage(v_Wind,C_leakage):
    if (v_Wind<0.25):
        return 0.25*C_leakage
    else:
        return v_Wind*C_leakage

#Phương trình 14
def def_f_VentForced(Eta_InsScr,U_VentForced,Phi_VentForced,A_Flr):
    return Eta_InsScr*U_VentForced*Phi_VentForced/A_Flr

#Phương trình 13
def def_f_VentSide(Eta_InsScr,ff_VentSide,f_leakage,U_ThScr,f_VentRoofSide,Eta_Side,Eta_SideThr):
    if (Eta_Side>=Eta_SideThr):
        return Eta_InsScr*ff_VentSide+0.5*f_leakage
    else:
        return Eta_InsScr*(U_ThScr*ff_VentSide+(1-U_ThScr)*f_VentRoofSide*Eta_Side)+0.5*f_leakage

#Phương trình 9
def def_MC_AirOut(f_VentSide,f_VentForced,CO2_Air,CO2_Out):
    return (f_VentSide+f_VentForced)*(CO2_Air-CO2_Out)

#Phương trình 17
def def_ff_VentRoof(C_d,U_Roof,A_Roof,A_Flr,g,h_Roof,Delta_T_AirOut,T_mean,C_w,v_Wind):
    component1=C_d*U_Roof*A_Roof/(2*A_Flr)
    component2=(g*h_Roof*Delta_T_AirOut/(2*T_mean))+C_w*(v_Wind**2)
    return component1*(component2**(1/2))
#Phương trình 16
def def_f_VentRoof(Eta_InsScr,ff_VentRoof,f_leakage,U_ThScr,f_VentRoofSide,Eta_Roof,Eta_RoofThr):
    if (Eta_Roof>=Eta_RoofThr):
        return Eta_InsScr*ff_VentRoof+0.5*f_leakage
    else:
        return Eta_InsScr*(U_ThScr*ff_VentRoof+(1-U_ThScr)*f_VentRoofSide*Eta_Roof)+0.5*f_leakage

#Phương trình 15
def def_MC_TopOut(f_VentRoof,CO2_Top,CO2_Out):
    return f_VentRoof*(CO2_Top-CO2_Out)

#Phương trình 19
def def_h_C_Buf(C_Buf,C_Max_Buf):
    if (C_Buf>C_Max_Buf):
        return 0
    else:
        return 1

#Phương trình 18
def def_MC_AirCan(M_CH2O,h_C_Buf,P,R_HOHAP):
    return M_CH2O*h_C_Buf*(P-R_HOHAP)



Các hàm phần Tốc độ

In [5]:
#Phương trình tính P
def def_P(J,CO2_Stom,Gamma):
    component1=J*(CO2_Stom-Gamma)
    component2=4*(CO2_Stom+2*Gamma)
    return component1/component2

#Phương trình tính J
def def_J(J_POT,anpha,PAR_Can,Theta):
    component1=J_POT+anpha*PAR_Can
    component2=4*Theta*J_POT*anpha*PAR_Can
    return (component1-((component1**2)-component2)**(1/2))/(2*Theta)

#Phương trình tính J_POT
def def_J_POT(J_Max_Can,E_j,T_Can,T_25,R,S,H):
    component1=J_Max_Can*math.exp(E_j*(T_Can-T_25)/(R*T_Can*T_25))
    component2=1+math.exp((S*T_25-H)/(R*T_25))
    component3=1+math.exp((S*T_Can-H)/(R*T_Can))
    return component1*component2/component3

#Phương trình tính J_Max_Can
def def_J_Max_Can(LAI,J_Max_Leaf):
    return LAI*J_Max_Leaf

#Phương trình tính CO2_Stom
def def_CO2_Stom(Eta_CO2_Air_Stom,CO2_Air):
    return Eta_CO2_Air_Stom*CO2_Air

#Phương trình tính Gamma
def def_Gamma(c_Gamma,T_Can):
    return (1/LAI)*c_Gamma*T_Can+20*c_Gamma*(1-1/LAI)



Lấy dữ liệu cho các biến cần lấy dữ liệu

In [6]:
DATA1=pd.read_csv("https://raw.githubusercontent.com/CEAOD/Data/master/GH_Cucumber/AutonomousGreenhouseChallengeFirstEdition(2018)/Sonoma/Greenhouse_climate.csv")
DATA2=pd.read_csv("https://raw.githubusercontent.com/CEAOD/Data/master/GH_Cucumber/AutonomousGreenhouseChallengeFirstEdition(2018)/meteo.csv")
df1=pd.DataFrame(DATA1)
df2=pd.DataFrame(DATA2)
#df1

In [7]:
df=pd.DataFrame()
df[['GHtime','CO2air','Tair','VentLee','Ventwind']]=df1[['GHtime','CO2air','Tair','VentLee','Ventwind']]
df[['Tout','Windsp']]=df2[['Tout','Windsp']]
df['GHtime']=pd.TimedeltaIndex(df['GHtime'],unit='d')+dt.datetime(1899,12,30)

(df['GHtime'][1]-df['GHtime'][0]).total_seconds()

df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
type((df['GHtime'][2]-df['GHtime'][1]).total_seconds())
print(df)

                             GHtime       CO2air  Tair    VentLee  Ventwind  \
0     2018-08-14 00:10:00.000048000   427.000000  19.9  52.900000       2.9   
1     2018-08-14 00:15:00.000028799   443.000000  19.9  52.900000       2.9   
2     2018-08-14 00:20:00.000009600   443.999999  19.9  51.099999       2.9   
3     2018-08-14 00:24:59.999990399   435.000001  19.7  42.800001       2.9   
4     2018-08-14 00:29:59.999971200   442.000000  19.9  42.800000       2.9   
...                             ...          ...   ...        ...       ...   
32985 2018-12-07 00:34:59.999952000   952.999999  24.5   0.000000       0.0   
32986 2018-12-07 00:40:00.000019200   900.000004  24.5   0.000000       0.0   
32987 2018-12-07 00:45:00.000000000   965.000000  24.7   0.000000       0.0   
32988 2018-12-07 00:49:59.999980799  1169.999989  24.6   0.000000       0.0   
32989 2018-12-07 00:55:00.000048000  1001.000023  24.7   0.000000       0.0   

       Tout  Windsp  
0      17.7     3.2  
1      

Tham số đã biết

In [8]:
cap_CO2_Air=3.8 #m
cap_CO2_Top=0.4 #m 
A_Flr=14000 #m^2
Eta_HeatCO2=0.057 #mg{CO2}*J^(-1)
P_Blow=500 #W
Phi_ExtCO2=72000 #mg*s^(-1)
CO2_Out=668 #mg*m^(-3)
Phi_Pad=0 #? #m^3*s^(-1)
K_ThScr=0.05*(10**(-3)) #m*K^(-2/3)*s^(-1)
g=9.81 #m*s^(-1)
Delta_T_AirTop= 1 #K
RO_Mean= 1.2 #kg*m^(-3) 
Delta_RO=0.0004837  #kg*m^(-3)

A_Roof=1400  #m^2
A_Side=1400 #m^2
C_d=0.75 #no unit
C_w=0.09 #no unit
h_SideRoof=1.9 #m
h_Roof=0.68 #m
Delta_T_AirOut= 5 #K
T_mean=298 #K
Zeta_InsScr=1 #no unit
C_leakage=1*(10**(-4)) #no unit
Eta_SideThr=0.9 #no unit
Eta_RoofThr=0.9 #no unit
Phi_VentForced= 0 #m^3*s^(-1)
C_Max_Buf=20*(10**3)  #mg{CO2}*m^(-2)
C_Buf=10*(10**3)  #mg{CO2}*m^(-2)
M_CH2O=0.03 #mg(μmol)^-1

#Quang hợp
LAI=3 #no unit
c_Gamma=1.7 #µmol{CO2}/mol{air}
Eta_CO2_Air_Stom=0.67 #µmol{CO2}/mol{air}
J_Max_Leaf=210 #μmol{e−}/m2.s
E_j=37000 #J/mol
T_25=298.15 #K
R=8.314 #J/mol.K
S=710 #J/mol.K
H=220000 #J/mol
anpha=0.385 #μmol{e−}/μmol{photons}
PAR_Can=1800 #μmol{photons}/m^2.s
Theta=0.7 #no unit
##valve
U_Blow=1 #no unit
U_ExtCO2=0.47 #no unit
U_Pad=1 #no unit
U_ThScr=0.56 #no unit
U_Roof=0.9 #no unit
U_Side=0.9 #no unit
U_VentForced=1 #no unit


CÁC BIẾN THAY ĐỔI SAU MỖI VÒNG LẶP, BAN ĐẦU KHỞI TẠO LÀ 0.



In [9]:
MC_BlowAir=0
MC_ExtAir=0
MC_PadAir=0
MC_AirCan=0
MC_AirTop=0
MC_AirOut=0
MC_TopOut=0
CO2_Air=0
CO2_Top=0
T=0
Eta_Side=0
Eta_Roof=0
v_Wind=0
T_Can=0

CHẠY SOLVER EULER

In [10]:
#Lấy giá trị CO2 tại thời điểm bất kỳ sau khi đã loại bỏ các hàng có giá trị NaN (dropna)))
#Dòng lấy giá trị
Start=943 
#Số lượng test.
NumberTest=10
#Chuyển đổi đơn vị của CO2Air từ ppm sang mg/m^3
CO2_Air=float(ConvertAirFrom_PPM_To_mgDivm3(df['CO2air'][Start]))
#Giả định CO2_Air=CO2_Top
CO2_Top=CO2_Air
#Số lượng bước nhảy trong mỗi 5 phút
distance=300
#Đưa vào mảng LisOfResult là mảng lưu giá trị sau mỗi thời điểm để chạy giải thuật Euler
ListOfResult=[[CO2_Air,CO2_Top]]
#Chạy chương trình với khoảng thời gian là NumberTest*distance (s)) 
for i in range(0,(NumberTest)*distance):
    # Lấy kết quả CO2air và CO2Top gần nhất để tính toán dx 
    CO2_Air=ListOfResult[-1][0]
    CO2_Top=ListOfResult[-1][1]
    # Lấy kết quả các biến thay đổi liên tục. Do nếu giá trị thời điểm đó chưa có giá trị trong data thì ta sẽ 
    # lấy giá trị ở thời điểm 5 phút gần nhất phía trước nên ta sẽ dùng int(i/distance))
    T_Air=float(df['Tair'][int(i/distance)+Start]+273)
    T_Can=T_Air+1 #giả định
    Eta_Side=float(df['Ventwind'][int(i/distance)+Start]/100)
    Eta_Roof=float(df['VentLee'][int(i/distance)+Start]/100)
    v_Wind=float(df['Windsp'][int(i/distance)+Start])
    #Tính giá trị hàm dx
    res_dx=dx(cap_CO2_Air,cap_CO2_Top,MC_BlowAir,MC_ExtAir,MC_PadAir,MC_AirCan,MC_AirTop,MC_AirOut,MC_TopOut)
    #Vì khoảng cách tính giữa 2 bước là 1s nên cho delta_T=1
    delta_T=1
    #Áp dụng công thức Euler
    CO2_next=[res_dx[0]*delta_T+ListOfResult[-1][0],res_dx[1]*delta_T+ListOfResult[-1][1]]
    ListOfResult.append(CO2_next)
#Lưu kết quả dự đoán vào biến predict
Predict=[]
for i in range(0,len(ListOfResult)):
    if (i%distance==0):
        Predict.append(ListOfResult[i][0])
        #print(ListOfResult[i])
#Lưu kết quả thực tế vào biến actual
Actual=[]
for i in range(Start,Start+NumberTest+1):
    Actual.append(float(ConvertAirFrom_PPM_To_mgDivm3(df['CO2air'][i])))
#Một vài giá trị dự đoán
print("Sau 5 phút:")
print("Thực tế",Actual[1],"(mg/m^3)")
print("Dự đoán",Predict[1],"(mg/m^3)")
print("Sau 15 phút:")
print("Thực tế",Actual[3],"(mg/m^3)")
print("Dự đoán",Predict[3],"(mg/m^3)")
print("Sau 40 phút:")
print("Thực tế",Actual[8],"(mg/m^3)")
print("Dự đoán",Predict[8],"(mg/m^3)")
print("Sau 50 phút:")
print("Thực tế",Actual[10],"(mg/m^3)")
print("Dự đoán",Predict[10],"(mg/m^3)")
#Tính sai số theo Mean absolute error từ phút thứ 5 đến phút thứ 50
#Giá trị đầu lấy từ data nếu không tính là dự đoán
MAE=0
for i in range(1,NumberTest+1):
    MAE=MAE+abs(Predict[i]-Actual[i])
MAE=MAE/NumberTest
#Kết quả sai số  trung bình cả đoạn theo Mean absolute error
print("Mean absolute square=",MAE,"(mg/m^3)")


Sau 5 phút:
Thực tế 1007.7759947507432 (mg/m^3)
Dự đoán 1011.8620567893935 (mg/m^3)
Sau 15 phút:
Thực tế 1061.7640000000001 (mg/m^3)
Dự đoán 1074.5112977122546 (mg/m^3)
Sau 40 phút:
Thực tế 1113.9524025944597 (mg/m^3)
Dự đoán 1130.843459879435 (mg/m^3)
Sau 50 phút:
Thực tế 1189.535600181009 (mg/m^3)
Dự đoán 1137.2981433337154 (mg/m^3)
Mean absolute square= 20.44662127605678 (mg/m^3)
